In [ ]:
using Pkg
Pkg.activate("../../environments/refined-delaunay-for-flow-problems/");

In [ ]:
using LinearAlgebra
using Random
using Statistics

using AbstractPlotting
using CairoMakie
using LightGraphs
using JuMP
using Parameters
using SCIP
using Triangle

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = 0.95 * WIDTH * rand(N)
y = 0.95 * HEIGHT * rand(N)
points = [x y];

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

In [ ]:
function draw_points!(points; markersize=4, color=:black)
    scatter!(points[:, 1], points[:, 2], markersize=markersize, color=color)
end

In [ ]:
make_scene()
draw_points!(points)

In [ ]:
struct Triangulation
    points::Matrix{Float64}   # n x 2
    edges::Matrix{Int64}      # m x 2
    triangles::Matrix{Int64}  # t x 3
end

In [ ]:
function unique_edges(triangles)
    set = Set()
    for t in 1:size(triangles, 1)
        triangle = triangles[t, :]
        push!(set, min(triangle[[1, 2]], triangle[[2, 1]]))
        push!(set, min(triangle[[2, 3]], triangle[[3, 2]]))
        push!(set, min(triangle[[1, 3]], triangle[[3, 1]]))
    end
    return hcat(sort(collect(set))...)'
end

In [ ]:
function delaunay_triangulation(points)::Triangulation
    points_map = collect(1:size(points, 1))
    triangle_array = Triangle.basic_triangulation(points, points_map)
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(points, edges, triangles)
end

In [ ]:
function draw_edges!(triangulation; color=:gray)
    @unpack points, edges = triangulation
    linesegments!(points[edges'[:], :], color=color)
end

In [ ]:
function draw_triangulation(triangulation)
    make_scene()
    draw_edges!(triangulation)
    draw_points!(triangulation.points)   
end

In [ ]:
del = delaunay_triangulation(points)
draw_triangulation(del)

In [ ]:
pointset_mean(array) = dropdims(mean(array, dims=2), dims=2)

In [ ]:
abstract type TriangleCenter end

struct TriangleCentroid <: TriangleCenter end
struct TriangleIncenter <: TriangleCenter end
struct TriangleFermatPoint <: TriangleCenter end

In [ ]:
function triangle_centers(triangulation, ::TriangleCentroid)
    @unpack points, triangles = triangulation
    return pointset_mean(points[triangles, :])
end

triangle_centers(t) = triangle_centers(t, TriangleCentroid())

In [ ]:
function triangle_centers(triangulation, ::TriangleIncenter)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        corners = points[t, :]
        a = norm(corners[2, :] - corners[3, :])
        b = norm(corners[1, :] - corners[3, :])
        c = norm(corners[1, :] - corners[2, :])
        
        incenter = [a b c] * corners ./ (a + b + c)
        push!(centers, incenter)
    end
    
    return vcat(centers...)
end

In [ ]:
function fermat_point(corners::Matrix{Float64})
    @assert size(corners) == (3, 2)
    l, u = extrema(corners)
    
    model = direct_model(SCIP.Optimizer(display_verblevel=4))
    @variable(model, l ≤ x ≤ u)
    @variable(model, l ≤ y ≤ u)
    @variable(model, len[1:3] ≥ 0.0)
    @variable(model, dx[1:3])
    @variable(model, dy[1:3])
    @constraint(model, ddx[i in 1:3], dx[i] == x - corners[i, 1])
    @constraint(model, ddy[i in 1:3], dy[i] == y - corners[i, 2])
    @constraint(model, soc[i in 1:3], [len[i], dx[i], dy[i]] in SecondOrderCone())
    @objective(model, Min, sum(len[i] for i in 1:3))
    
    optimize!(model)
    return value.([x y])
end

function triangle_centers(triangulation, ::TriangleFermatPoint)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        push!(centers, fermat_point(points[t, :]))
    end
    return vcat(centers...)
end

In [ ]:
fermat_point(Float64[0 0; 0 1; 1 0])

In [ ]:
draw_triangulation(del)
draw_points!(triangle_centers(del), color=:limegreen)
draw_points!(triangle_centers(del, TriangleIncenter()), color=:magenta)
# draw_points!(triangle_centers(del, TriangleFermatPoint()), color=:navy)

In [ ]:
function delaunay_with_centers(triangulation, method=TriangleCentroid())
    centers = triangle_centers(triangulation, method)
    all_points = vcat(triangulation.points, centers)
    return delaunay_triangulation(all_points)
end

In [ ]:
draw_triangulation(delaunay_with_centers(del))

In [ ]:
draw_triangulation(delaunay_with_centers(del, TriangleIncenter()))

In [ ]:
draw_triangulation(delaunay_with_centers(delaunay_with_centers(del)))

In [ ]:
function constrained_with_centers(triangulation, method=TriangleCentroid())
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation, method)
    all_points = vcat(points, centers)
    point_map = collect(1:size(all_points, 1))
    
    triangle_array = Triangle.constrained_triangulation(all_points, point_map, edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(all_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_with_centers(del))

In [ ]:
draw_triangulation(constrained_with_centers(constrained_with_centers(del)))

In [ ]:
function edge_midpoints(triangulation)
    @unpack points, edges = triangulation
    return pointset_mean(points[edges, :])
end

In [ ]:
draw_triangulation(del)
draw_points!(edge_midpoints(del), color=:red)

In [ ]:
# triangulate with edge subdivision, again
del_ = delaunay_triangulation(vcat(del.points, edge_midpoints(del)))
draw_triangulation(delaunay_triangulation(vcat(del_.points, edge_midpoints(del_))))

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 1)
        edge = edges[e, :]
        push!(set, [edge[1] e + offset])
        push!(set, [e + offset edge[2]])
    end
    return vcat(set...)
end

In [ ]:
function constrained_subdivision(triangulation)
    @unpack points, edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation))
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_subdivision(del))

In [ ]:
draw_triangulation(constrained_subdivision(constrained_subdivision(del)))

In [ ]:
# alternate: first add triangle centers, then subdivide edges
draw_triangulation(constrained_subdivision(constrained_with_centers(del)))

In [ ]:
# alternate 2: first subdivide edges, the add triangle centers
draw_triangulation(constrained_with_centers(constrained_subdivision(del)))

In [ ]:
# alternate 2.5: first subdivide edges, the add triangle centers (unconstrained)
draw_triangulation(delaunay_with_centers(constrained_subdivision(del)))

In [ ]:
function constrained_combined_refinement(triangulation, method=TriangleCentroid())
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation, method)
    new_points = vcat(points, edge_midpoints(triangulation), centers)
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_combined_refinement(del))

In [ ]:
draw_triangulation(constrained_combined_refinement(constrained_combined_refinement(del)))

# Steiner Tree Model

In [ ]:
function antiparallel_digraph(triangulation)
    @unpack points, edges = triangulation
    graph = SimpleDiGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
        add_edge!(graph, t, s)
    end
    return graph
end

In [ ]:
function edge_lengths(points, edges)
    diff = points[edges[:, 1], :] - points[edges[:, 2], :]
    return dropdims(mapslices(norm, diff, dims=2), dims=2)
end

edge_lengths(triangulation) = edge_lengths(triangulation.points, triangulation.edges)

In [ ]:
function edge_length_map(triangulation)
    @unpack edges = triangulation
    lengths = edge_lengths(triangulation)
    
    length_map = Dict{Tuple{Int64, Int64}, Float64}()
    for e = 1:size(edges, 1)
        s, t = edges[e, :]
        length_map[s, t] = lengths[e]
        length_map[t, s] = lengths[e]
    end
    return length_map
end

In [ ]:
function steiner_tree(triangulation, terminals)
    # Compute length by Euclidean distance of nodes.
    lengths = edge_length_map(triangulation)
    
    # Build digraph with all antiparallel arcs, for nonnegative flow.
    graph = antiparallel_digraph(triangulation)
    nodes = collect(1:nv(graph))
    arcs = collect(keys(lengths))
    
    length(terminals) >= 2 || error("Need at least 2 terminals.")
    all(terminals .<= nv(graph)) || error("Terminals out of range.")
    root = terminals[1]
    sinks = terminals[2:end]
   
    demand(v, s) = 1.0*(v == s) - 1.0*(v == root)
    
    # Using arc length for fixed capacity cost and multi-commodity flow.
    model = JuMP.direct_model(SCIP.Optimizer(display_verblevel=0))
    @variable(model, select[a in arcs], Bin, container=SparseAxisArray)
    @variable(model, flow[a in arcs,s in sinks] ≥ 0, container=SparseAxisArray)
    @constraint(model, balance[v in nodes, s in sinks],
        sum(flow[(n, v), s] - flow[(v, n), s] for n in neighbors(graph, v))
        == demand(v, s))
    @constraint(model, capacity[a in arcs, s in sinks], flow[a, s] <= select[a])
    @objective(model, Min, sum(lengths[a] * select[a] for a in arcs))
    
    optimize!(model)
    
    return objective_value(model), value.(select)

end

In [ ]:
function draw_tree(triangulation, terminals)
    @unpack points, edges = triangulation

    
    obj, select = steiner_tree(triangulation, terminals)
    @show obj
    selected = [select[(s,t)] + select[(t,s)] for (s,t) in eachrow(edges)]
    active_edges = edges[selected .> 0.0, :]
    
    draw_triangulation(triangulation)
    linesegments!(points[active_edges'[:], :], color=:plum, linewidth=3)
end

In [ ]:
terminals = collect(1:7);

In [ ]:
draw_tree(del, terminals)

In [ ]:
draw_tree(constrained_subdivision(del), terminals)

In [ ]:
draw_tree(delaunay_with_centers(del), terminals)

In [ ]:
draw_tree(constrained_with_centers(del, TriangleIncenter()), terminals)

In [ ]:
draw_tree(constrained_combined_refinement(del, TriangleIncenter()), terminals)

# Nearest Neighbors

In [ ]:
dump(Triangulation)

In [ ]:
#  1 2
#      5
#  4 3
cross = Triangulation(
    [0.0 200.0; 200.0 200.0; 200.0 0.0; 0.0 0.0; 400.0 100.0],
    [1 3; 2 4; 2 3; 2 5; 3 5],
    [2 3 5; ],
)

In [ ]:
draw_triangulation(cross)

In [ ]:
draw_triangulation(constrained_subdivision(cross))

In [ ]:
# Does not really keep all edges.
# For crossing edges, we should compute intersection point and add it manually?

In [ ]:
points

In [ ]:
graph, dists = euclidean_graph(points'[:,:], cutoff=200.0)
@show graph dists;

In [ ]:
function edges_from_graph(graph)
    rows = []
    for edge in edges(graph)
        push!(rows, [src(edge) dst(edge)])
    end
    return vcat(rows...)
end

In [ ]:
nn = Triangulation(points, edges_from_graph(graph))